# Set up our libraries

In [37]:
import time
import pandas as pd
import numpy as np
import re
import nltk
from datetime import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

nltk.download('stopwords') #download the latest stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bryant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# import our data

df = pd.read_csv('mergedProjectsAbstracts.csv',encoding='utf-8-sig')

print(df.shape)

/Users/bryant/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1040239, 26)


# Part 1. Project Terms Contain 'Opioid'

The fedreporter data have a project_terms field. A subset of projects
contain 'opioid' in that field. Let's make a flag for that.


In [60]:
st = time.time()

df['projectTermsFlag'] = pd.to_numeric(np.where(
    df['PROJECT_TERMS'].str.contains("opioid", case=False, na=False), 1, ''))

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - st)))

In [71]:
print('% flagged: ' + str(100*df['projectTermsFlag'].value_counts()[1]/len(df)))

% flagged: 0.8569184581620185


# Part 2. 'Wiki Approach'

If we read through the [opioid wikipedia page](https://en.wikipedia.org/wiki/Opioid) we can extract words to highlight in project abstracts that might indicate it is a project related to opioids.

In [72]:
# terms/n-grams gleaned from wikipedia
opioid_terms = ['opioid','opiate','morphine','heroin',
                'percocet','vicoprofen','dextromethorphan','loperamide',
                'naloxegol','hydrocodone','oxycodone','fentanyl',
                'naloxone','analgesics','carfentanil','benzodiazepines',
                'narcotic','opium','cocaine','codeine',
                'pain relief','cancer pain','anesthesia','chronic pain',
                'nerve pain','fibromyalgia','overdose','addiction',
                'withdrawal','dependence','recreational use','euphoria',
                'tolerance','controlled substance','over-prescription',
                'peripheral nervous system','psychoactive','agonist',
                'antagonist','blood-brain']

In [78]:
# clean up the abstracts for text matching
st = time.time()

# a regex pattern to help eliminate punctuation
nonchars = re.compile( r'\W+|\d+' )

# function to elim punctuation and set to lower case
def clean(text):
    return re.sub(nonchars, " ", text).lower()

# cut the end coding that interferes with cleaning function
# df = df[0:-1]

# drop nulls, as they don't help our analysis
df_denull = df[df.ABSTRACT.notnull()]

# may have to drop weird end coding that interferes with cleaning function
# df_denull = df_denull[0:-1]

df_denull['cleanText'] = df_denull['ABSTRACT'].apply(clean)

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - st)))

00:06:10


/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [80]:
st = time.time()

# Count the appearances of our defined terms in each abstract
def countTerm(text):
    return len(re.findall(term,text))

for term in opioid_terms:
    df_denull[term] = df_denull['cleanText'].apply(countTerm)
    
# sum of all term frequencies by abstract
df_denull['sumTermCounts'] = df_denull[opioid_terms].sum(axis=1)

# set term threshold
wikiThreshold = 2;

def wikiFlag(row):
    if row['sumTermCounts'] > wikiThreshold:
        return 1
    else:
        return None
    
df_denull['wikiTermsFlag'] = df_denull.apply(wikiFlag, axis=1)

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - st)))

/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


00:46:06


/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [89]:
df_denull['wikiTermsFlag'].dtype

dtype('float64')

# Part 3. Text analysis (Topic modeling)

LDA (Latent Dirichlet Allocation) is a model used for discovering abstract topics from a collection of documents. These 'latent' topics can be discovered based on observed data -- words in the documents, in this case.

To surface these topics, we create a matrix where each document is a row and each column is a word in the corpus vocabulary. The corpus vocabulary is the universe of words present in any one or more documents in the corpus (minus chosen 'stopwords' that we consider to provide little information).

Each cell of the matrix would be a count of that word in that document. A variant increases the level of sophistication by using a normalized version of these counts known as TF-IDF. TF stands for term-frequency and TF-IDF is term-frequency times inverse document-frequency. In other words, we are not only looking for how often a word appears in a given document, but also whether this particular word is distinct across all the collections of documents (corpus). For example, intuitively we understand that words like "often" or "use" are more frequently encountered, but they are less informative (more semantically-vacuous) if we want to discern a particular topic of a document, as they might be frequently encounter across all text documents in a corpus. On the other hand, words which we will see less frequently across a collection of document might indicate that those words are specific to a particular document, and, therefore, constitute a basis for a topic.

We provide the model with this matrix and how many topics we want it to use. Think of it like a k-means clustering analog. The model will then iterate a specified number of times considering two distributions; 1) which words in the vocabulary are more or less probable to belong in a given topic and 2) which topic is more or less probable for a given document.

The main assumptions, if this all went over your head:
* each document consists of a mixture of topics, and
* each topic consists of a collection of words.


In [38]:
# prepare a df of just abstracts
abstracts = df[['PROJECT_ID', 'ABSTRACT']]
print(abstracts.shape)

(1040239, 2)


In [39]:
# drop nulls
trunc_abstracts = abstracts.dropna()
print(trunc_abstracts.shape)

(1032895, 2)


In [46]:
# set up our corpus (limit number of rows while perfecting code)
df_modeling = trunc_abstracts[:1000]
# df_modeling = trunc_abstracts

# Get only the text of abstracts
corpus = df_modeling['ABSTRACT'].values.tolist()

In [47]:
# stop words and stemming them
stemmer = nltk.SnowballStemmer('english')
eng_stopwords = stopwords.words('english')
add_stopwords = ['understand','method']
comb_stopwords = eng_stopwords + add_stopwords
stemmed_stopwords = []

for w in comb_stopwords:
    stemmed_stopwords.append(stemmer.stem(w))

In [49]:
start_time = time.time()
'''
Before we can apply LDA, we need to create a vocabulary with all the words in our data
We specify to only include those words that appear in less than 10% of the document 
and appear in at least 0.5% of documents. 
docs: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

The exercise of running the vectorizer on each year independently, then only
keeping ngrams that all 10 years of abstracts possess is all geared towards breaking up
the computation into manageable chunks that won't crash the kernel.
'''

# prepare our vectorizer
analyzer = CountVectorizer().build_analyzer()

# our stemming function
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(analyzer=stemmed_words,
                                  max_df=0.10,
                                  min_df=0.005,
#                                   max_features = 1500, # useful for limiting coderun duration
                                  ngram_range = (0,2),
                                  stop_words=stemmed_stopwords)

doc_term_matrix = stem_vectorizer.fit_transform(corpus)
doc_term_features = stem_vectorizer.get_feature_names()

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))


00:00:03


In [50]:
'''
Using below code, we can print out the many, many words excluded due to:
- occurred in too many documents (max_df)
- occurred in too few documents (min_df)
- were cut off by feature selection (max_features)
'''

# print(count_vect.stop_words_)

'\nUsing below code, we can print out the many, many words excluded due to:\n- occurred in too many documents (max_df)\n- occurred in too few documents (min_df)\n- were cut off by feature selection (max_features)\n'

In [52]:
start_time = time.time()

# Use LDA to create topics.

LDA = LatentDirichletAllocation(n_components=10, random_state=1)  
LDA.fit(doc_term_matrix)

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))


00:00:03


In [54]:
# initialize list for topics
topicList = []

# fill out topics list as top 20 words in each topic
for i,topic in enumerate(LDA.components_):
    ithTopic = [doc_term_features[i] for i in topic.argsort()[-20:]]
    topicList.append(ithTopic)
    
topicListDf = pd.DataFrame(topicList)
# topicListDf.head()

In [55]:
start_time = time.time()

'''
Assign the probability of all the topics to each document, then
add a column to the original data frame that will store the highest-scoring
topic for that abstract.
'''

# matrix where each row is an abstract, each column a topic. Each cell is value of that topic for that abstract.
topic_values = LDA.transform(doc_term_matrix)  

'''
take the column number associated with the highest value in a given row, store in our analytical dataframe
'''
df_modeling['primeTopicId'] = topic_values.argmax(axis=1)

'''
store the valence of that prime topic for that abstract as well
'''
df_modeling['primeTopicValence'] = topic_values.max(axis=1)

print("Shape of topic_values: ",(topic_values.shape))

print(time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))


Shape of topic_values:  (1000, 10)
00:00:39


/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# which topics are most common among projects tagged explicitly?
pd.DataFrame(df_modeling[df_modeling.tagCompare == 'both'].primeTopicId.value_counts())[:10]

In [36]:
start_time = time.time()

# Convert a collection of raw documents to a matrix of TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(abstracts_list)

# Get feature names
vectorizer_feature_names = vectorizer.get_feature_names()

# Run the model with 10 topics
nmf = NMF(n_components=5, random_state=1).fit(tfidf)

nmf_W = nmf.transform(tfidf) # get topics to documents matrix
nmf_H = nmf.components_ # get word to topics matrix

nmf_W[0]

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

00:00:03


In [31]:
# View the list of topics (10 top words per topic)

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

Topic 0:
----------------------------
indicators research environmental project species water land ecosystem ecological objective
----------------------------
Topic 1:
----------------------------
toxic air metal emissions trace disseminate releases element worldwide implementation
----------------------------
Topic 2:
----------------------------
pm particles effects exposure health particulate studies air matter ambient
----------------------------
Topic 3:
----------------------------
process treatment soil water waste project design technology contaminated organic
----------------------------
Topic 4:
----------------------------
ozone lung exposure cells ntp air standard ppm effects function
----------------------------


In [20]:
# View a top document related to a given topic

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(abstracts_list[doc_index])

--------------------
Topic 0:
--------------------
protein dna gene proteins genes genetic expression genome rna molecular
--------------------
DESCRIPTION (provided by applicant): The regulation of gene expression is vital for healthy development and physiology, and many diseases are caused by or associated with changes in gene expression. In the past decade, a tremendous amount of information has been gathered regarding transcriptional control by transcription factors that bind directly to regulatory DNA sequences in or around their target genes. In addition, microRNAs that control gene expression post-transcriptionally have been studied extensively. However, functional and biochemical information about the vast majority of RNA binding proteins has been lacking despite clear evidence of their importance in development and disease. A major factor contributing to our limited understanding of post- transcriptional control by RNA binding proteins is a shortage of appropriate technologies

--------------------
The SEER Program was initiated in 1972 in response to requirements of the National Cancer Program for assessing the magnitude of the cancer burden in the United States, and for identifying factors related to cancer risk and/or patient survival. The SEER Program has among its objectives: 1. 	To assemble and report, on a periodic basis, estimates of cancer incidence, especially among the following key cancer sites: breast cancer, lung cancer, colorectal cancer, prostate cancer, pancreatic cancer, and urinary bladder cancer.2. 	To monitor annual cancer incidence trends to identify unusual changes in specific forms of cancer occurring in population subgroups defined by geographic, demographic, and social characteristics. 3. 	To provide continuing information on changes over time in extent of disease at diagnosis, trends in therapy, and changes in cancer patient survival. 4. 	To identify the occurrence of possible iatrogenic cancers, i.e., cancers that are caused by can

cells cell immune stem tumor signaling mice il responses aim
--------------------
White blood cells called T lymphocytes play critical roles in immune defense against viruses, bacteria, fungi, protozoa, and cancer cells. In the unactivated state, these cells circulate in the blood and accumulate in lymphoid tissues such as lymph nodes and spleen. Upon encounter with foreign materials (antigens) on the membranes of specialized antigen presenting cells (dendritic cells), these resting T-cells become activated, undergo numerous cell divisions, and differentiate into effector cells. The effector cells leave the lymphoid tissues and blood, entering sites of infection to combat pathogens. They can also invade normal tissues where their activity can cause autoimmune pathology. After elimination of an infecting organism, most of the activated T-cells die, but some remain as memory cells, to provide a more rapid and vigorous response if the same pathogen is encountered in the future. Recent wor

In [ ]:
print('Time to run whole notebook: ')
elapsed_time = time.time() - nb_start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))